### In our last model in file "Music_Generation_Train1.ipynb", we got only 82% accuracy. However, in order to generate melodious music, we need at least 90% accuracy. 
### So, we have loaded the weights of last epoch from our previous model into this model and also we have added 2 extra layers of LSTM here with more LSTM units. 
### Here, we are fine-tuning our old layers and we have added more layers. In short, here we are doing "Transfer Learning" from old to new model.

In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

Using TensorFlow backend.


In [9]:
data_directory = "../Data2/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = '../Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [10]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i] + 1. 
        yield X, Y

In [11]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    model.load_weights("../Data/Model_Weights/Weights_80.h5", by_name = True)
    
    return model

In [12]:
def training_model(data, epochs = 90):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../Data2/log.csv", index = False)

In [13]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             44544     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 256)             0         
_________________________________________________________________
lstm_3 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_3 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_4 (LSTM)                (16, 64, 256)             525312    
___________________________________________________________

/home/ishdutt/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Batch: 1, Loss: 4.467709064483643, Accuracy: 0.0068359375
Batch: 2, Loss: 4.44322395324707, Accuracy: 0.1728515625
Batch: 3, Loss: 4.3970208168029785, Accuracy: 0.134765625
Batch: 4, Loss: 4.27142333984375, Accuracy: 0.1025390625
Batch: 5, Loss: 3.859605073928833, Accuracy: 0.1435546875
Batch: 6, Loss: 3.5472381114959717, Accuracy: 0.166015625
Batch: 7, Loss: 3.4820306301116943, Accuracy: 0.1259765625
Batch: 8, Loss: 3.606796979904175, Accuracy: 0.083984375
Batch: 9, Loss: 3.67295503616333, Accuracy: 0.091796875
Batch: 10, Loss: 3.474689483642578, Accuracy: 0.1435546875
Batch: 11, Loss: 3.209568977355957, Accuracy: 0.1796875
Batch: 12, Loss: 3.4669735431671143, Accuracy: 0.154296875
Batch: 13, Loss: 3.6918156147003174, Accuracy: 0.130859375
Batch: 14, Loss: 3.4363884925842285, Accuracy: 0.1494140625
Batch: 15, Loss: 3.6404342651367188, Accuracy: 0.1240234375
Batch: 16, Loss: 3.416712999343872, Accuracy: 0.1240234375
Batch: 17, Loss: 3.336678981781006, Accuracy: 0.1494140625
Batch: 18, 

Batch: 141, Loss: 2.67238187789917, Accuracy: 0.2724609375
Batch: 142, Loss: 2.616115093231201, Accuracy: 0.2900390625
Batch: 143, Loss: 2.7206220626831055, Accuracy: 0.2587890625
Batch: 144, Loss: 2.6209723949432373, Accuracy: 0.283203125
Batch: 145, Loss: 2.5609865188598633, Accuracy: 0.3046875
Batch: 146, Loss: 2.7138280868530273, Accuracy: 0.2763671875
Batch: 147, Loss: 2.7063002586364746, Accuracy: 0.2822265625
Batch: 148, Loss: 2.6564769744873047, Accuracy: 0.287109375
Batch: 149, Loss: 2.6704626083374023, Accuracy: 0.275390625
Batch: 150, Loss: 2.562335968017578, Accuracy: 0.2978515625
Batch: 151, Loss: 2.6264989376068115, Accuracy: 0.2744140625
Epoch 2/90
Batch: 1, Loss: 2.6199207305908203, Accuracy: 0.30859375
Batch: 2, Loss: 2.3902170658111572, Accuracy: 0.3388671875
Batch: 3, Loss: 2.5615901947021484, Accuracy: 0.3017578125
Batch: 4, Loss: 2.6339166164398193, Accuracy: 0.2705078125
Batch: 5, Loss: 2.466928005218506, Accuracy: 0.3349609375
Batch: 6, Loss: 2.37581729888916, Ac

Batch: 130, Loss: 2.212592601776123, Accuracy: 0.3896484375
Batch: 131, Loss: 2.056767463684082, Accuracy: 0.4345703125
Batch: 132, Loss: 2.103119134902954, Accuracy: 0.41796875
Batch: 133, Loss: 2.035266876220703, Accuracy: 0.451171875
Batch: 134, Loss: 2.012732982635498, Accuracy: 0.4423828125
Batch: 135, Loss: 1.9215540885925293, Accuracy: 0.474609375
Batch: 136, Loss: 1.9799778461456299, Accuracy: 0.4345703125
Batch: 137, Loss: 1.8057197332382202, Accuracy: 0.45703125
Batch: 138, Loss: 1.7365813255310059, Accuracy: 0.490234375
Batch: 139, Loss: 1.8087605237960815, Accuracy: 0.4833984375
Batch: 140, Loss: 1.9627230167388916, Accuracy: 0.4443359375
Batch: 141, Loss: 1.9445555210113525, Accuracy: 0.462890625
Batch: 142, Loss: 1.975831151008606, Accuracy: 0.43359375
Batch: 143, Loss: 2.0716018676757812, Accuracy: 0.4130859375
Batch: 144, Loss: 1.9568816423416138, Accuracy: 0.4462890625
Batch: 145, Loss: 1.8707671165466309, Accuracy: 0.4501953125
Batch: 146, Loss: 2.121628761291504, Acc

...

Batch: 10, Loss: 0.2843887209892273, Accuracy: 0.904296875
Batch: 11, Loss: 0.2808610796928406, Accuracy: 0.8994140625
Batch: 12, Loss: 0.280459463596344, Accuracy: 0.90234375
Batch: 13, Loss: 0.27668797969818115, Accuracy: 0.9052734375
Batch: 14, Loss: 0.3196190595626831, Accuracy: 0.884765625
Batch: 15, Loss: 0.2681213915348053, Accuracy: 0.9033203125
Batch: 16, Loss: 0.28538385033607483, Accuracy: 0.9072265625
Batch: 17, Loss: 0.2695368528366089, Accuracy: 0.91015625
Batch: 18, Loss: 0.25518712401390076, Accuracy: 0.9189453125
Batch: 19, Loss: 0.2884845435619354, Accuracy: 0.8984375
Batch: 20, Loss: 0.2700183391571045, Accuracy: 0.9208984375
Batch: 21, Loss: 0.26330921053886414, Accuracy: 0.9111328125
Batch: 22, Loss: 0.3243558704853058, Accuracy: 0.8896484375
Batch: 23, Loss: 0.27841827273368835, Accuracy: 0.90234375
Batch: 24, Loss: 0.30882418155670166, Accuracy: 0.888671875
Batch: 25, Loss: 0.28743815422058105, Accuracy: 0.90625
Batch: 26, Loss: 0.25621944665908813, Accuracy: 0.9

Batch: 148, Loss: 0.314567506313324, Accuracy: 0.8916015625
Batch: 149, Loss: 0.2722565233707428, Accuracy: 0.912109375
Batch: 150, Loss: 0.28808751702308655, Accuracy: 0.908203125
Batch: 151, Loss: 0.28292983770370483, Accuracy: 0.908203125
Epoch 88/90
Batch: 1, Loss: 0.41124504804611206, Accuracy: 0.8740234375
Batch: 2, Loss: 0.351108580827713, Accuracy: 0.8798828125
Batch: 3, Loss: 0.28752216696739197, Accuracy: 0.9091796875
Batch: 4, Loss: 0.27728772163391113, Accuracy: 0.9091796875
Batch: 5, Loss: 0.2660360336303711, Accuracy: 0.9140625
Batch: 6, Loss: 0.2623897194862366, Accuracy: 0.90234375
Batch: 7, Loss: 0.27442091703414917, Accuracy: 0.908203125
Batch: 8, Loss: 0.3035637140274048, Accuracy: 0.88671875
Batch: 9, Loss: 0.33678752183914185, Accuracy: 0.87890625
Batch: 10, Loss: 0.23748081922531128, Accuracy: 0.9208984375
Batch: 11, Loss: 0.30242282152175903, Accuracy: 0.8955078125
Batch: 12, Loss: 0.2915058135986328, Accuracy: 0.8994140625
Batch: 13, Loss: 0.2770404815673828, Ac

Batch: 134, Loss: 0.2760615944862366, Accuracy: 0.90625
Batch: 135, Loss: 0.2670942544937134, Accuracy: 0.9140625
Batch: 136, Loss: 0.2833065390586853, Accuracy: 0.9052734375
Batch: 137, Loss: 0.33825892210006714, Accuracy: 0.8857421875
Batch: 138, Loss: 0.2881850004196167, Accuracy: 0.8984375
Batch: 139, Loss: 0.3137814700603485, Accuracy: 0.884765625
Batch: 140, Loss: 0.3009709417819977, Accuracy: 0.8984375
Batch: 141, Loss: 0.32912492752075195, Accuracy: 0.8935546875
Batch: 142, Loss: 0.28759652376174927, Accuracy: 0.908203125
Batch: 143, Loss: 0.2465699166059494, Accuracy: 0.9033203125
Batch: 144, Loss: 0.29952237010002136, Accuracy: 0.90625
Batch: 145, Loss: 0.27078181505203247, Accuracy: 0.9091796875
Batch: 146, Loss: 0.3178447484970093, Accuracy: 0.9013671875
Batch: 147, Loss: 0.29617825150489807, Accuracy: 0.9013671875
Batch: 148, Loss: 0.3048643469810486, Accuracy: 0.8916015625
Batch: 149, Loss: 0.25577402114868164, Accuracy: 0.9111328125
Batch: 150, Loss: 0.3048265874385834, 

Batch: 121, Loss: 0.298291951417923, Accuracy: 0.8935546875
Batch: 122, Loss: 0.2794665992259979, Accuracy: 0.912109375
Batch: 123, Loss: 0.26393407583236694, Accuracy: 0.9150390625
Batch: 124, Loss: 0.30159008502960205, Accuracy: 0.8984375
Batch: 125, Loss: 0.3239758312702179, Accuracy: 0.89453125
Batch: 126, Loss: 0.3277030885219574, Accuracy: 0.890625
Batch: 127, Loss: 0.2610899806022644, Accuracy: 0.908203125
Batch: 128, Loss: 0.29228436946868896, Accuracy: 0.900390625
Batch: 129, Loss: 0.23380601406097412, Accuracy: 0.916015625
Batch: 130, Loss: 0.2903742492198944, Accuracy: 0.90234375
Batch: 131, Loss: 0.2714284062385559, Accuracy: 0.9169921875
Batch: 132, Loss: 0.3105519413948059, Accuracy: 0.90234375
Batch: 133, Loss: 0.2711535692214966, Accuracy: 0.8974609375
Batch: 134, Loss: 0.29843318462371826, Accuracy: 0.8955078125
Batch: 135, Loss: 0.28851938247680664, Accuracy: 0.9052734375
Batch: 136, Loss: 0.300447940826416, Accuracy: 0.8955078125
Batch: 137, Loss: 0.3469092845916748,

Batch: 108, Loss: 0.28905144333839417, Accuracy: 0.8994140625
Batch: 109, Loss: 0.27493003010749817, Accuracy: 0.908203125
Batch: 110, Loss: 0.2752034664154053, Accuracy: 0.912109375
Batch: 111, Loss: 0.31388071179389954, Accuracy: 0.8994140625
Batch: 112, Loss: 0.28292393684387207, Accuracy: 0.9013671875
Batch: 113, Loss: 0.26326093077659607, Accuracy: 0.91796875
Batch: 114, Loss: 0.2736370861530304, Accuracy: 0.9033203125
Batch: 115, Loss: 0.32232531905174255, Accuracy: 0.8935546875
Batch: 116, Loss: 0.28056493401527405, Accuracy: 0.908203125
Batch: 117, Loss: 0.3078760504722595, Accuracy: 0.892578125
Batch: 118, Loss: 0.249839186668396, Accuracy: 0.9248046875
Batch: 119, Loss: 0.23525449633598328, Accuracy: 0.91796875
Batch: 120, Loss: 0.30413201451301575, Accuracy: 0.9033203125
Batch: 121, Loss: 0.30030953884124756, Accuracy: 0.896484375
Batch: 122, Loss: 0.30062806606292725, Accuracy: 0.900390625
Batch: 123, Loss: 0.29793936014175415, Accuracy: 0.9091796875
Batch: 124, Loss: 0.294

In [14]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,2.626499,0.274414
1,2,1.945449,0.461914
2,3,1.626868,0.541016
3,4,1.439776,0.564453
4,5,1.354747,0.580078
5,6,1.257122,0.612305
6,7,1.174994,0.619141
7,8,1.128270,0.639648
8,9,1.057377,0.657227
9,10,1.025230,0.667969
